In [6]:
import pickle
import numpy as np
import pandas as pd
import sys
sys.path.append('src')

**Generate one dataset**

In [ ]:
%cd options_utils

In [ ]:
!pip install --no-cache-dir -e .


> **Generate reference prices**



In [ ]:
!python optimal_stopping/run/run_algo.py --configs=config1_prices --nb_jobs=2;


> **Generate the sample paths**



In [ ]:
!python optimal_stopping/run/generate_paths.py --configs=config1_paths --nb_jobs=2;

In [ ]:
df_ = pd.read_csv('output/metrics_draft/config1_prices.csv')
df = df_[['hurst','price']]
prices = df.groupby('hurst', as_index=False)['price'].mean()
paths  = pickle.load(open('output/metrics_draft/config1_paths.obj','rb'))

**Run one toy experiment**

In [ ]:
%cd src

In [8]:
from KES import model
from DR_RBF import model as model_RBF
from DR_matern import model as model_mat

In [ ]:
paths = paths[15:-15]
prices = prices.iloc[15:-15]

In [ ]:
sub = 3
M = 500 
paths_sub = [p[:M,:10,:] for p in paths[::sub]]
scores,stdv, results = model_RBF(paths_sub, prices['price'].to_numpy()[::sub],at=True,ll=None,cv=3, mode='krr',NUM_TRIALS=3)

100%|██████████| 3/3 [12:35<00:00, 251.68s/it]


In [ ]:
print('matern', scores,stdv)

matern 0.0027480169133565096 0.0030575108330338154


In [ ]:
print('RBF', scores,stdv)

RBF 0.0010070803628937787 0.0007484485197370775


In [ ]:
sub = 3
M = 500
alphas0 = [0.1,1,10]
alphas1 = [1] # there is no alphas1 for MMD0
lambdas = [1] # there is no lambdas for MMD0
#0.01,0.5,1.,5.,10.,20.
paths_sub = [p[:M,:10,:] for p in paths[::sub]]
scores,stdv, results, ind_train, ind_test, model_ = model(paths_sub, prices['price'].to_numpy()[::sub],rank=0, alphas0=alphas0,alphas1=alphas1,lambdas=lambdas,at=True,ll=None,cv=3, mode='krr',imbalanced=False,NUM_TRIALS=3)

In [ ]:
print('MMD0', scores, stdv)

MMD0 0.0010055297396127782 0.0004128193724958566


In [ ]:
sub = 3
M = 500
alphas0 = [1]
alphas1 = [1]
lambdas = [0.1,1,10] 
paths_sub = [p[:M,:10,:] for p in paths[::sub]]
scores,stdv, results, ind_train, ind_test, model_ = model(paths_sub, prices['price'].to_numpy()[::sub],rank=1, alphas0=alphas0,alphas1=alphas1,lambdas=lambdas,at=True,ll=None,cv=3, mode='krr',imbalanced=False,NUM_TRIALS=3)

In [ ]:
print('MMD1', scores, stdv)

MMD1 0.000531886298712851 0.00022121269197428498
